In [1]:
import itertools as it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy.misc
import scipy.optimize as op
import scipy.io
import random
from IPython.display import display, Math, Latex

# set path
import os
os.chdir('%s/courses/coursera_bigdata/course3/week2' % os.getenv('DST'))
print(os.getcwd())

# printing options 
np.set_printoptions(linewidth=250)
np.set_printoptions(precision=3)

%matplotlib inline
# plotting options
plt.rc('font'  , size=18)
plt.rc('figure', figsize=(10, 8))
plt.rc('axes'  , labelsize=22)
plt.rc('legend', fontsize=16)
plt.rc('figure', figsize=(10, 8))

/Users/rwk7t/Development/dst/courses/coursera_bigdata/course3/week2


# Import data

In [2]:
def munge(df):
    df.columns    = [c.replace(' ', '_').lower() for c in df.columns]
    df.start_date = pd.to_datetime(df['start_date'])
    df.end_date   = pd.to_datetime(df['end_date'  ])
    return 

In [3]:
df1 = pd.read_csv('bike_data/201402_trip_data.csv'); munge(df1)
df2 = pd.read_csv('bike_data/201408_trip_data.csv'); munge(df2)
df3 = pd.read_csv('bike_data/201508_trip_data.csv'); munge(df3)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144015 entries, 0 to 144014
Data columns (total 11 columns):
trip_id              144015 non-null int64
duration             144015 non-null int64
start_date           144015 non-null datetime64[ns]
start_station        144015 non-null object
start_terminal       144015 non-null int64
end_date             144015 non-null datetime64[ns]
end_station          144015 non-null object
end_terminal         144015 non-null int64
bike_#               144015 non-null int64
subscription_type    144015 non-null object
zip_code             137885 non-null object
dtypes: datetime64[ns](2), int64(5), object(4)
memory usage: 13.2+ MB


In [5]:
pd.to_datetime(df1.head(4)['start_date'])

0   2013-08-29 14:13:00
1   2013-08-29 14:42:00
2   2013-08-29 10:16:00
3   2013-08-29 11:29:00
Name: start_date, dtype: datetime64[ns]

In [6]:
min(df1.start_date), min(df2.start_date), min(df3.start_date)

(Timestamp('2013-08-29 09:08:00'),
 Timestamp('2014-03-01 00:14:00'),
 Timestamp('2014-09-01 00:05:00'))

In [7]:
min(df1.end_date), min(df2.end_date), min(df3.end_date)

(Timestamp('2013-08-29 09:11:00'),
 Timestamp('2014-03-01 00:17:00'),
 Timestamp('2014-09-01 00:14:00'))

In [8]:
max(df1.start_date), max(df2.start_date), max(df3.start_date)

(Timestamp('2014-02-28 23:20:00'),
 Timestamp('2014-08-31 22:31:00'),
 Timestamp('2015-08-31 23:26:00'))

In [9]:
max(df1.end_date), max(df2.end_date), max(df3.end_date)

(Timestamp('2014-03-02 16:52:00'),
 Timestamp('2014-09-01 16:09:00'),
 Timestamp('2015-08-31 23:39:00'))

In [10]:
df = pd.concat([df1, df2, df3])
df.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
0,520,63,2013-08-29 14:14:00,South Van Ness at Market,66,2013-08-29 14:13:00,South Van Ness at Market,66,NaN,Subscriber,4576,94127
1,661,70,2013-08-29 14:43:00,San Jose City Hall,10,2013-08-29 14:42:00,San Jose City Hall,10,NaN,Subscriber,4607,95138
2,48,71,2013-08-29 10:17:00,Mountain View City Hall,27,2013-08-29 10:16:00,Mountain View City Hall,27,NaN,Subscriber,4130,97214


In [11]:
del(df1)
del(df2)
del(df3)

# Quesiton 1:

Utilizing the Bay Area Bike Share database - what is the most popular start station based on trip data?

In [12]:
q1 = df.groupby('start_station', sort=True).count().sort('duration', ascending=False)
q1.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
start_station,,,,,,,,,,,
San Francisco Caltrain (Townsend at 4th),49092,49092,49092,49092,49092,49092,49092,39254,9838,49092,48916
San Francisco Caltrain 2 (330 Townsend),33742,33742,33742,33742,33742,33742,33742,28766,4976,33742,33658
Harry Bridges Plaza (Ferry Building),32934,32934,32934,32934,32934,32934,32934,25591,7343,32934,32312


In [13]:
q1.head(1).index.values[0]

'San Francisco Caltrain (Townsend at 4th)'

# Question 2:

Utilizing the Bay Area Bike Share database (Year 1 only, Aug. 2013- Feb 1, 2014) - Which is the least popular(least used) start station in the Bike share trips data?

(Hint: Use the count of start station, group and order in ascending order)

In [14]:
q2 = df.query('start_date < "3/1/2014"').groupby('start_station', sort=True).count().sort('duration', ascending=True)
q2.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
start_station,,,,,,,,,,,
Mezes Park,3,3,3,3,3,3,3,0,3,3,3
San Jose Government Center,23,23,23,23,23,23,23,0,23,23,22
Redwood City Public Library,44,44,44,44,44,44,44,0,44,44,41


# Question 3:

Utilizing the Bay Area Bike Share database Year 1 - what is the Second most popular end station based on trip data ?

In [15]:
q3 = df.query('start_date < "3/1/2014 00:00"').groupby('end_station', sort=True).count().sort('duration', ascending=False)
q3.head(3)

,bike_#,duration,end_date,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
end_station,,,,,,,,,,,
San Francisco Caltrain (Townsend at 4th),11637,11637,11637,11637,11637,11637,11637,0,11637,11637,11466
Embarcadero at Sansome,7590,7590,7590,7590,7590,7590,7590,0,7590,7590,6659
Harry Bridges Plaza (Ferry Building),7475,7475,7475,7475,7475,7475,7475,0,7475,7475,6991


In [16]:
q3.iloc[1]

bike_#               7590
duration             7590
end_date             7590
end_terminal         7590
start_date           7590
start_station        7590
start_terminal       7590
subscriber_type         0
subscription_type    7590
trip_id              7590
zip_code             6659
Name: Embarcadero at Sansome, dtype: int64